In [447]:
import pandas as pd
from datetime import datetime
charging_sessions = pd.read_csv('charging_sessions2.csv')
bump_avril = pd.read_csv('bump_avril.csv')
bump_mai = pd.read_csv('bump_mai.csv')

In [448]:
bump_avril.columns = bump_avril.iloc[21]
bump_avril = bump_avril.iloc[22:-6,:]
bump_avril.reset_index(drop=True, inplace=True)
bump_avril['Date']= pd.to_datetime(bump_avril['Date'].astype(str), infer_datetime_format=True)

bump_mai.columns = bump_mai.iloc[21]
bump_mai = bump_mai.iloc[22:-6,:]
bump_mai.reset_index(drop=True, inplace=True)
bump_mai['Date']= pd.to_datetime(bump_mai['Date'].astype(str), infer_datetime_format=True)

bump = pd.concat([bump_avril, bump_mai], axis = 0)

In [449]:
charging_sessions['start_date']= pd.to_datetime(charging_sessions['start_date'].astype(str), infer_datetime_format=True)
charging_sessions['end_date']= pd.to_datetime(charging_sessions['end_date'].astype(str), infer_datetime_format=True)

bump['Durée (min)'] = bump['Durée (min)'].str.replace(",", "")
bump['Durée (min)'] = pd.to_numeric(bump['Durée (min)'])
bump['Energie (kWh)'] = bump['Energie (kWh)'].str.replace(",", "")
bump['Energie (kWh)'] = pd.to_numeric(bump['Energie (kWh)'])

In [450]:
charging_sessions = charging_sessions.rename(columns={"site": "Site"})

In [451]:
print(charging_sessions[charging_sessions['Site']!="NO MATCH"]['booking_id'].nunique())

3772


In [452]:
charging_sessions['day'] = charging_sessions['start_date'].dt.month
bump['day'] = bump['Date'].dt.month
bump['bump_duration_delta_time'] = bump['Durée (min)'].astype(float).apply(pd.to_timedelta, unit = 'm')
bump['bump_end_date'] = bump['Date'] + bump['bump_duration_delta_time']

In [453]:
charging_sessions.columns

Index(['Unnamed: 0', 'plate_number', 'model', 'start_date', 'end_date',
       'start_battery', 'end_battery', 'adress', 'location', 'driver',
       'next_driver', 'mileage', 'booking_id', 'delta', 'month', 'day',
       'created_id', 'last_mileage', 'last_plate', 'distance', 'kWh',
       'kWh_used', 'battery_used', 'consumption', 'Site'],
      dtype='object')

In [454]:
grouped_bump = bump.groupby(by = ['Site', 'N° ']).agg({'N° ': 'first', 'Date': 'first', 'bump_end_date': 'first', 'day': "first",'Energie (kWh)': 'first', 'Borne': 'first', 'Durée (min)': 'first'})
grouped_bump

N°   \
Site                                               N°           
BMP - Campanile - Berges du Rhône - Lyon | 4-6 ... 1        1   
                                                   10      10   
                                                   100    100   
                                                   101    101   
                                                   102    102   
...                                                       ...   
Total/TOTAL-NF059983 | 1 Rue de Strasbourg         2999  2999   
                                                   3000  3000   
                                                   3001  3001   
                                                   3002  3002   
                                                   3003  3003   

                                                                       Date  \
Site                                               N°                         
BMP - Campanile - Berges du Rhône - Lyon | 4-6 ... 1    2022-04-03 08:15:13   
                                                   10   2022-04-01 18:10:19   
                                                   100  2022-04-25 23:47:58   
                                                   101  2022-04-25 13:22:54   
                                                   102  2022-04-24 23:15:08   
...                                                                     ...   
Total/TOTAL-NF059983 | 1 Rue de Strasbourg         2999 2022-05-03 19:34:20   
                                                   3000 2022-05-02 12:41:29   
                                                   3001 2022-05-02 20:06:19   
                                                   3002 2022-05-03 12:17:38   
                                                   3003 2022-05-04 12:25:34   

                                                                  bump_end_date  \
Site                                               N°                             
BMP - Campanile - Berges du Rhône - Lyon | 4-6 ... 1    2022-04-03 10:08:53.200   
                                                   10   2022-04-01 19:39:56.800   
                                                   100  2022-04-26 01:15:14.800   
                                                   101  2022-04-25 15:02:46.800   
                                                   102  2022-04-25 00:42:26.000   
...                                                                         ...   
Total/TOTAL-NF059983 | 1 Rue de Strasbourg         2999 2022-05-03 20:18:20.000   
                                                   3000 2022-05-02 13:27:29.000   
                                                   3001 2022-05-02 21:07:19.000   
                                                   3002 2022-05-03 13:02:38.000   
                                                   3003 2022-05-04 12:49:34.000   

                                                         day  Energie (kWh)  \
Site                                               N°                         
BMP - Campanile - Berges du Rhône - Lyon | 4-6 ... 1     4.0         24.400   
                                                   10    4.0         22.500   
                                                   100   4.0         22.500   
                                                   101   4.0         26.000   
                                                   102   4.0         22.500   
...                                                      ...            ...   
Total/TOTAL-NF059983 | 1 Rue de Strasbourg         2999  5.0         16.760   
                                                   3000  5.0         25.048   
                                                   3001  5.0         25.797   
                                                   3002  5.0         27.510   
                                                   3003  5.0         16.482   

                                                                            

In [455]:
grouped_sessions = charging_sessions.groupby(by = ['Site', 'booking_id']).agg({'start_date': 'first', 'end_date': 'first', 'day': 'first', 'kWh': 'first', 'booking_id': 'first'})


In [456]:
charging_sessions.columns

Index(['Unnamed: 0', 'plate_number', 'model', 'start_date', 'end_date',
       'start_battery', 'end_battery', 'adress', 'location', 'driver',
       'next_driver', 'mileage', 'booking_id', 'delta', 'month', 'day',
       'created_id', 'last_mileage', 'last_plate', 'distance', 'kWh',
       'kWh_used', 'battery_used', 'consumption', 'Site'],
      dtype='object')

In [457]:
merged_df = grouped_bump.merge(grouped_sessions, on=["Site", "day"])

In [458]:
merged_df = merged_df.rename(columns={"N° ": "BUMP_id", "Date": "BUMP_start_date", "bump_end_date": "BUMP_end_date", "Energie (kWh)": "BUMP_kWh"})                      



In [459]:
print(merged_df['BUMP_id'].nunique())
print(merged_df['booking_id'].nunique())
print(len(merged_df))

2224
2925
1100717


In [460]:
merged_df = merged_df[abs((merged_df.BUMP_start_date - merged_df.start_date) / pd.Timedelta(minutes=1)) < 500]
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 33338 entries, BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France to BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BUMP_id          33338 non-null  object        
 1   BUMP_start_date  33338 non-null  datetime64[ns]
 2   BUMP_end_date    33338 non-null  datetime64[ns]
 3   day              33338 non-null  float64       
 4   BUMP_kWh         33338 non-null  float64       
 5   Borne            23918 non-null  object        
 6   Durée (min)      33338 non-null  float64       
 7   start_date       33338 non-null  datetime64[ns]
 8   end_date         33338 non-null  datetime64[ns]
 9   kWh              33338 non-null  float64       
 10  booking_id       33338 non-null  object        
dtypes: datetime64[ns](4), float64(4), object(3)
memory usage

In [461]:
print(merged_df['BUMP_id'].nunique())
print(merged_df['booking_id'].nunique())

2217
2865


In [462]:
merged_df = merged_df[abs((merged_df.BUMP_end_date - merged_df.end_date) / pd.Timedelta(minutes=1)) < 500]
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25728 entries, BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France to BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BUMP_id          25728 non-null  object        
 1   BUMP_start_date  25728 non-null  datetime64[ns]
 2   BUMP_end_date    25728 non-null  datetime64[ns]
 3   day              25728 non-null  float64       
 4   BUMP_kWh         25728 non-null  float64       
 5   Borne            18890 non-null  object        
 6   Durée (min)      25728 non-null  float64       
 7   start_date       25728 non-null  datetime64[ns]
 8   end_date         25728 non-null  datetime64[ns]
 9   kWh              25728 non-null  float64       
 10  booking_id       25728 non-null  object        
dtypes: datetime64[ns](4), float64(4), object(3)
memory usage

In [463]:
merged_df = merged_df[abs(merged_df['BUMP_kWh'].astype(float) - merged_df['kWh']) < 3]
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8643 entries, BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France to BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   BUMP_id          8643 non-null   object        
 1   BUMP_start_date  8643 non-null   datetime64[ns]
 2   BUMP_end_date    8643 non-null   datetime64[ns]
 3   day              8643 non-null   float64       
 4   BUMP_kWh         8643 non-null   float64       
 5   Borne            6431 non-null   object        
 6   Durée (min)      8643 non-null   float64       
 7   start_date       8643 non-null   datetime64[ns]
 8   end_date         8643 non-null   datetime64[ns]
 9   kWh              8643 non-null   float64       
 10  booking_id       8643 non-null   object        
dtypes: datetime64[ns](4), float64(4), object(3)
memory usage:

In [464]:
merged_df

,BUMP_id,BUMP_start_date,BUMP_end_date,day,BUMP_kWh,Borne,Durée (min),start_date,end_date,kWh,booking_id
Site,,,,,,,,,,,
"BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France",186,2022-04-24 10:43:01,2022-04-24 12:35:11.800,4.0,28.00,None,112.18,2022-04-24 02:55:35,2022-04-24 05:25:52,28.00,RZTY9073622
"BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France",186,2022-04-24 10:43:01,2022-04-24 12:35:11.800,4.0,28.00,None,112.18,2022-04-24 07:12:15,2022-04-24 12:45:20,25.92,RZTY9074111
"BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France",186,2022-04-24 10:43:01,2022-04-24 12:35:11.800,4.0,28.00,None,112.18,2022-04-24 12:43:15,2022-04-24 14:58:40,27.00,RZTY9074814
"BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France",187,2022-04-24 22:18:45,2022-04-25 00:02:07.200,4.0,24.30,None,103.37,2022-04-24 14:57:26,2022-04-25 00:26:28,23.76,RZTY9075601
"BMP - Paripark - Jardin des Plantes - Paris | 15 Rue Censier, 75005 Paris, France",187,2022-04-24 22:18:45,2022-04-25 00:02:07.200,4.0,24.30,None,103.37,2022-04-25 00:19:43,2022-04-25 01:59:28,25.38,RZTY9078116
...,...,...,...,...,...,...,...,...,...,...,...
"BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France",2402,2022-05-01 01:38:19,2022-05-01 03:14:26.800,5.0,25.00,AVE#1 - BMP - IES S/N000142,96.13,2022-05-01 03:20:00,2022-05-01 09:53:00,26.19,RZTY9108784
"BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France",2403,2022-05-30 19:52:25,2022-05-30 20:50:38.800,5.0,16.60,AVE#2 - BMP - IES S/N000146,58.23,2022-05-30 21:51:00,2022-05-31 00:59:00,18.63,RZTY9270015
"BMP - Parking AVE - 17e | 7 Rue Christine de Pisan, 75017 Paris, France",2406,2022-05-01 04:03:43,2022-05-01 15:48:19.000,5.0,37.69,AVE#1 - BMP - IES S/N000142,704.60,2022-05-01 06:03:00,2022-05-01 12:46:00,35.20,RZTY9109118


In [465]:
merged_df.drop(['Durée (min)', 'day'], axis = 1, inplace = True)

In [466]:
print(merged_df['BUMP_id'].nunique())
print(merged_df['booking_id'].nunique())

1796
2127


In [467]:
merged_df.to_csv("merged_sessions.csv")

In [468]:
bump.columns

Index([                       nan,                      'N° ',
                           'Date',              'RFID tag ID',
                           'Site',            'Energie (kWh)',
                    'Durée (min)',         'Prix Unitaire HT',
                          'Unité',            'Prix Total HT',
                           'Type',                        nan,
                          'Borne',                      'day',
       'bump_duration_delta_time',            'bump_end_date'],
      dtype='object', name=21)

In [520]:
bump['week'] = bump['Date'].dt.isocalendar().week
charging_sessions['week'] = charging_sessions['start_date'].dt.isocalendar().week

In [522]:
grouped_bump = bump.groupby(by=['Site', 'week']).agg({'Energie (kWh)': 'sum'})
grouped_sessions = charging_sessions.groupby(by=['Site', 'week']).agg({'kWh': 'sum'})

In [523]:
merged_df_sum = pd.merge(grouped_bump, grouped_sessions, on = ['Site', 'week'])

In [524]:
merged_df_sum.to_csv("kWh_totals.csv")
merged_df_sum = pd.read_csv("kWh_totals.csv")

In [530]:
merged_df_sum

,Site,week,BUMP kWh,BOOKINGS kWh,difference (kWh)
0,BMP - Paripark - Jardin des Plantes - Paris | ...,13.0,691,903,211
1,BMP - Paripark - Jardin des Plantes - Paris | ...,14.0,2542,2618,76
2,BMP - Paripark - Jardin des Plantes - Paris | ...,15.0,2101,1840,261
3,BMP - Paripark - Jardin des Plantes - Paris | ...,16.0,2403,2259,143
4,BMP - Paripark - Jardin des Plantes - Paris | ...,17.0,2320,2281,39
5,BMP - Paripark - Jardin des Plantes - Paris | ...,18.0,2316,2187,128
6,BMP - Paripark - Jardin des Plantes - Paris | ...,19.0,2404,2149,254
7,BMP - Paripark - Jardin des Plantes - Paris | ...,20.0,2789,2240,549
8,BMP - Paripark - Jardin des Plantes - Paris | ...,21.0,2785,2245,540
9,BMP - Paripark - Jardin des Plantes - Paris | ...,22.0,799,1090,291


In [526]:
merged_df_sum = merged_df_sum.rename(columns={"Energie (kWh)": "BUMP kWh", "kWh": "BOOKINGS kWh"})

In [527]:
merged_df_sum = merged_df_sum.append({'Site': 'Total', "BUMP kWh": merged_df_sum['BUMP kWh'].sum(), "BOOKINGS kWh": merged_df_sum['BOOKINGS kWh'].sum()}, ignore_index=True)


In [528]:
merged_df_sum['difference (kWh)'] = abs(merged_df_sum['BUMP kWh']-merged_df_sum['BOOKINGS kWh'])

In [529]:
merged_df_sum['difference (kWh)'] = merged_df_sum['difference (kWh)'].astype(int)
merged_df_sum['BUMP kWh'] = merged_df_sum['BUMP kWh'].astype(int)
merged_df_sum['BOOKINGS kWh'] = merged_df_sum['BOOKINGS kWh'].astype(int)

In [531]:
merged_df_sum.to_csv("kWh_totals.csv")